In [1]:
import musicbox.theory as th
import musicbox.box as box
import random
import mido

In [2]:
roman = ['I', 'II', 'III', 'IV', 'V', 'VI', 'VII']
tonleiter = list(th.Tone.all())
progression = [['I', 'V', 'IV', 'I'],
              ['I', 'IV', 'V', 'IV'],
              ['I', 'I', 'IV', 'V'],
              ['I', 'IV', 'I', 'V'],
              ['I', 'IV', 'V', 'V'],
              ['I', 'VI', 'II', 'V'],
              ['I', 'VI', 'IV', 'V'],
              ['I', 'IV', 'V', 'I'],
              ['I', 'V', 'I'],
              ['VI', 'II', 'V', 'I']]

def setting():
    out = dict()
    out['root'] = random.choice(list(th.Tone.all()))
    scale_types = andom.choice(['major', 'minor'])
    out['scale_type'] = random.choice(scale_types)
    out['scale'] = th.Scale(out['root'], out['scale_type'])
    out['rhythm'] = [True for _ in range(20)]
    out['progression'] = random.choice(progression)
    return out

def select(tones, notes, previous=None):
    actual_notes = []
    for note in notes:
        for tone in tones:
            if note.tone == tone:
                actual_notes.append(note)
    return random.choice(actual_notes)

In [3]:
asdf = setting()
print(asdf['root'])
print(asdf['scale_type'])
print(repr(asdf['scale']))
print(asdf['rhythm'])
print(asdf['progression'])


G#
major
Scale("G#", "major")
[True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True]
['I', 'VI', 'II', 'V']


In [4]:
s = asdf['scale']
notes = []
for p in asdf['progression']:
    for r in asdf['rhythm']:
        notes.append(select(s.degree(p).tones, box.GI30.notes))

In [5]:
mid = mido.MidiFile()
track = mido.MidiTrack()
mid.tracks.append(track)
names = mido.get_output_names()
print(names)
for name in names:
    if 'FLUID' in name:
        port = mido.open_output(name)

['FLUID Synth (30402):Synth input port (30402:0) 129:0', 'Midi Through:Midi Through Port-0 14:0']


In [6]:
track.append(mido.Message('program_change', program=10, time=0))
# TODO set tempo bpm2tempo

In [7]:
for i, note in enumerate(notes):
    track.append(mido.Message('note_on', note=note.midi, velocity=64, time=100))
    track.append(mido.Message('note_off', note=note.midi, velocity=127, time=100))

In [8]:
for msg in mid.play():
    port.send(msg)

In [9]:
port.close()

In [10]:
# mid.save('test.mid')